# Predicted Business Loss Due to Churn

In this notebook, we estimate the loss of revenue to BigBank as a result of clients terminating their relationship with the bank.   

We use a model to determine the probability that each client will leave (a.k.a. churn), and calculate the projected loss to the business for each client using the churn probability and current revenue from the client.

## Setup

In [ ]:
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from pymongo import MongoClient

from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import train_test_split

from urth.widgets.widget_channels import channel

pd.set_option('display.precision', 2)
sns.set(style="whitegrid")

The following imports allow us to create [declarativewidgets](https://github.com/jupyter-incubator/declarativewidgets) widgets within the notebook to help us visualize and interact with our data.

In [ ]:
%%html
<link rel="import" href="urth_components/urth-viz-chart/urth-viz-chart.html" 
    is="urth-core-import">
<link rel="import" href="urth_components/urth-viz-table/urth-viz-table.html" 
    is="urth-core-import">
<link rel="import" href="urth_components/iron-flex-layout/classes/iron-flex-layout.html" 
    is='urth-core-import' package='PolymerElements/iron-flex-layout'>
<link rel='import' href='urth_components/paper-slider/paper-slider.html' 
    is='urth-core-import' package='PolymerElements/paper-slider'>
<link rel='import' href='urth_components/paper-card/paper-card.html' 
    is='urth-core-import' package='PolymerElements/paper-card'>
<link rel='import' href='urth_components/paper-item/paper-item.html' 
    is='urth-core-import' package='PolymerElements/paper-item'>
<link rel="import" href="urth_components/paper-dropdown-menu/paper-dropdown-menu.html" 
    is='urth-core-import' package='PolymerElements/paper-dropdown-menu'>
<link rel="import" href="urth_components/paper-menu/paper-menu.html"
    is='urth-core-import' package='PolymerElements/paper-menu'>

<urth-core-channel id='churnChannel' name='churn'></urth-core-channel>

## Load client data

Load information about BigBank clients.  The data consists of client metadata, such as age, gender, etc., as well as aggregate statistics about each client's banking activity (e.g., number of credit/debit card transactions, total transaction amount).

The data also include a `churn` classifier, which indicates whether or not the client is still a BigBank customer.

To load the data, modify `mongo_configs` with the appropriate IP address, port, username and password.

In [ ]:
mongo_configs = {
    "auth": {
        "port": 27017, 
        "password": "<mongoPass>", 
        "user": "<mongoUser>", 
        "host": "<mongoHost>",
        "db": "demo",
        "collection": "client_features"
    }, 
    "noauth": {
        "port": 27017, 
        "host": "<mongoHost>",
        "db": "demo",
        "collection": "client_features"
    }
}

These are helper functions to load the data from MongoDB and query a collection.

In [ ]:
def get_mongo_uri(**kwargs):
    if all([x in kwargs for x in ['user','password']]):
        return 'mongodb://{user}:{password}@{host}:{port}'.format(**kwargs)
    return 'mongodb://{host}:{port}'.format(**kwargs)

def query_collection(db, collection, limit=0):
    collection = db[collection]
    cursor = collection.find({}).limit(limit)
    df = pd.DataFrame(list(cursor))
    # Remove the MongoDB _id column
    del df['_id']
    return df

def load_data_from_mongo(uri, db_name, collection):
    client = MongoClient(uri)
    db = client[db_name]
    return query_collection(db, collection)

def load_data(location=None):
    loc = location or 'local'
    config = mongo_configs[loc]
    return load_data_from_mongo(
        get_mongo_uri(**config),
        config['db'],
        config['collection']
    )

In [ ]:
client_df = load_data()
client_df.head()

## Train churn model

Train a churn classifier, which we'll use to predict the probability that a client will churn.

To keep things simple in this example, we use a single data set, which we split it into training and test data sets.  We use the training data to train the model, and the test data to make projections about lost revenue to the bank.

In [ ]:
def make_feature_space(df):
    '''Create the feature space required by our classifier.'''
    # drop columns/features we don't want/need for the classifier
    features_df = df.drop(['churn', 'customer_id'], axis=1, errors='ignore')
    X = features_df.as_matrix().astype(np.float)
    # normalize feature values
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    return X

def predict_churn(X):
    '''Predict the probabilit of churn from feature set.'''
    return clf.predict_proba(X)[:,1]

def train_model(X, y):
    '''Train our classifier using features X and target variable y.'''
    clf = RF(n_estimators=100)
    return clf.fit(X, y)

def init_model(df):
    # split data into train, test sets
    train_index, test_index = train_test_split(df.index, random_state=99)
    train_df = client_df.ix[train_index]
    test_df = client_df.ix[test_index]

    # target variable
    y = np.array(train_df['churn'])

    # extract features
    X = make_feature_space(train_df)

    # train classifier
    clf = train_model(X, y)

    return clf, test_df

After training the model, we are left with the churn classifier and the test data set, which we'll use for our churn predictions.

In [ ]:
clf, test_df = init_model(client_df)
test_df.head()

## Calculate business loss

In this simple example, we calculate the projected loss of business (revenue) to BigBank for all clients in the test data set.   We calculate BigBank's revenue from each client, and multiply that by the churn probability to determine the predicted loss.

In [ ]:
def calc_business_loss(df):
    data = df[['customer_id']].copy()

    # extract features
    X = make_feature_space(df)
    
    # predict churn
    data['churn_probability'] = predict_churn(X)
    
    # TODO: avg_daily_balance would be a nice feature to have here
    # for now, we'll just use fraction of income
    avg_daily_balance = df['annual_income'] / 6

    # Interest made on deposits
    deposit_rate = 0.02

    # Fee collected for each credit txn
    credit_rate = 0.015

    # Assume we make some money on trading fees and/or portfolio management
    mgmt_rate = 0.02

    # How much is each customer worth to the business?
    worth = deposit_rate * avg_daily_balance + \
            mgmt_rate * df['annual_investment_rev'] + \
            credit_rate * df['total_txn_amount']
    data['worth'] = worth
    
    # How much would we lose per annum?
    data['predicted_loss'] = data['churn_probability'] * worth
    
    return data.sort_values(by='predicted_loss', ascending=False)

In [ ]:
churn_df = calc_business_loss(test_df)
churn_df.head()

Merge the predicted churn and business loss with the client data.

In [ ]:
data = test_df.merge(churn_df, on='customer_id', left_index=True, right_index=True)

## Loss by Age Group

Next, we visualize the projected loss of revenue by age group.  In our data set, age is an important feature in predicting if a client will churn.

In [ ]:
def predicted_loss_by_age(df):
    bin_size = 5
    _min, _max = int(df.age_years.min()), int(df.age_years.max())
    bins = range(_min, _max + bin_size, 5)
    by_age = df.groupby(pd.cut(df.age_years, bins=bins))
    return by_age['predicted_loss']

def plot_loss_by_age(df, how='sum'):
    loss_by_age = predicted_loss_by_age(df)
    if how == 'mean':
        data = loss_by_age.mean()
        title = 'Predicted Loss per Client'
    else:
        data = loss_by_age.sum()
        title = 'Total Predicted Loss'
    f, ax = plt.subplots(figsize=(12,8))
    ax = sns.barplot(x=data.index, y=data, ax=ax, color="#e74c3c")
    ax.set_title(title, size=13)
    ax.set_xlabel('Age group (years)', size=13)
    ax.set_ylabel('Predicted loss (US Dollars)', size=13)
    return ax

ax = plot_loss_by_age(data, how='mean')

## Widgets

Create widgets to visualize the predicted loss of business due to churn.

### Loss by age plot

We create a widget that displays the business loss by age group plot above.  The widget invokes the above function to generate and display the plot.  We add a drop-down widget to allow the user to select the method we use to aggregate the data.

In [ ]:
def w_plot_loss_by_age(how=None):
    ax = plot_loss_by_age(data, how)
    # Close the figure to keep matplotlib from rendering the plot;
    # it will be shown by the widget instead.
    plt.close()
    return ax.figure

In [ ]:
%%html
<template is="urth-core-bind" channel="churn">
    <div class="card-content">
        <paper-dropdown-menu label="Select aggregate method" 
                    selected-item-label="{{ by_age_method }}" noink>
            <paper-menu class="dropdown-content" selected="sum" 
                attr-for-selected="name">
                <paper-item name="sum">sum</paper-item>
                <paper-item name="mean">mean</paper-item>
            </paper-menu>
        </paper-dropdown-menu>
    </div>
    <urth-core-function
        ref="w_plot_loss_by_age"
        arg-how="{{ by_age_method }}"
        result="{{ by_age_plot }}" 
        auto></urth-core-function>
    <img src="{{ by_age_plot }}">
</template>

### Business loss scatter plot

We create a scatter plot widget that plots churn probability and projected loss of revenue.  

We use a slider widget to control the data to display in the plot.  In this case, the slider limits the results to clients who have the specified churn probability or greater.

In [ ]:
def churn(min_probability=None):
    '''Retrieve business loss for all clients with greater than `min_probability` of churn.
    '''
    if min_probability:
        probability = min_probability / 100. if min_probability > 1 else min_probability
    else:
        probability = 0
    df = data[data['churn_probability'] > probability]
    df = df.sort_values('predicted_loss', ascending=False)
    df.reset_index(drop=True, inplace=True)
    return df[['customer_id', 'worth', 'churn_probability', 'predicted_loss']]

In [ ]:
churn(90).head()

When we change the probability using the slider, the widget invokes the `churn` function to retrieve filtered results, which are passed to the scatter plot.

Each dot in the scatter plot represents a single client.  When a user clicks a dot, the client is "selected", and an event is sent over the widgets "channel".  Code or other widgets in our notebook can respond to this event.

In [ ]:
%%html
<template is="urth-core-bind" channel="churn">
    <h3 class="center horizontal layout">Predicted Loss of Business Due to Churn</h3>
    <div class="center horizontal layout">
        <div>Probability of Churn</div>
        <paper-slider min="5" max="95" step="5" pin snap value="{{ probability }}" editable></paper-slider>
        <div>%</div>
    </div>
    <urth-core-function 
        ref="churn"
        arg-min_probability="[[ probability ]]"
        result="{{ churn_scatter }}"
        limit="200"
        auto>
    </urth-core-function>
    <urth-viz-chart id="c5" type='scatter' datarows="{{ churn_scatter.data }}" 
        columns="{{ churn_scatter.columns }}" 
        selection="{{ selected }}"
        primary=3
        secondary=2
        xbounds="[0]"
        xlabel="Predicted Revenue Loss (US Dollars)"
        ylabel="Probability of Churn">
            <urth-viz-col index="2" type="numeric" format="%"></urth-viz-col>
    </urth-viz-chart>
</template>

### Client profile

We create a simple widget to show additional information about the selected client.  This widget updates when the user selects a client dot in the scatter plot widget above.  

On selection of each dot, the widgets framework updates the `selected` data, and our profile widget automatically invokes the `get_client_profile` function below with the `selected` data.  The function retrieves the full client record from our data set, formats some of the values, and returns the result to the widget, which updates its display.

In [ ]:
def _get_client_profile(customer_id):
    '''
    Lookup our customer from our client data.  
    Selectively choose which columns to include in profile.
    Format the values for display.
    '''
    df = data[data['customer_id'] == customer_id]
    if len(df) == 0:
        return {}
    client = df.to_dict(orient='records')[0]
    # Format for display
    for key in ['annual_income', 'annual_investment_rev', 'worth', 
                'predicted_loss', 'total_txn_amount', 'avg_txn_amount']:
        client[key] = '${:,.0f}'.format(client[key])
    client['age_years'] = int(client['age_years'])
    client['churn_probability'] = '{:d}%'.format(int(client['churn_probability'] * 100))
    return client

def get_client_profile(selected=[], columns=[]):
    if not selected:
        return
    
    # We only want to display a single client, so choose first 
    # row of selected data
    client = selected[0]
    if not client:
        return
    # Determine which column is customer_id in selected data 
    col = columns.index('customer_id')
    customer_id = client[col]
    # Lookup the client profile
    return _get_client_profile(customer_id)

In [ ]:
%%html
<template is="urth-core-bind" channel="churn">
    <urth-core-function ref="get_client_profile" 
        arg-selected="[[ selected ]]"
        arg-columns="[[ churn_scatter.columns ]]"
        result="{{ profile }}" auto></urth-core-function>
    <template is="dom-if" if="[[ profile ]]">
        <paper-card heading="ID: {{ profile.customer_id }}">
          <div class="card-content">
            <div class="layout horizontal justified">
                <span>Churn probability:</span><span>{{ profile.churn_probability }}</span></div>            
            <div class="layout horizontal justified">
                <span>Revenue:</span><span>{{ profile.worth }}</span></div>            
            <div class="layout horizontal justified">
                <span>Predicted loss:</span><span>{{ profile.predicted_loss }}</span></div>            
            <div class="layout horizontal justified">
                <span>Age:</span><span>{{ profile.age_years }}</span></div>            
            <div class="layout horizontal justified">
                <span>Income:</span><span>{{ profile.annual_income }}</span></div>            
            <div class="layout horizontal justified">
                <span>Investments:</span><span>{{ profile.annual_investment_rev }}</span></div>            
          </div>
        </paper-card>
    </template>
</template>

## Select data source

This is a simple drop-down widget to allow the notebook user to select from multiple backend MongoDB hosts.  When the user selects a new MongoDB location, it invokes a handler that reloads the client data from that location.  The handler also tickles variables on the widget channel, which triggers the widgets to refresh.

In [ ]:
def on_host_selected(old, new):
    host = new
    client_df = load_data(host)
    clf, test_df = init_model(client_df)
    churn_df = calc_business_loss(test_df)
    
    global data
    data = test_df.merge(churn_df, on='customer_id', left_index=True, right_index=True)
    
    # reset variables on the widget channel to force refresh of widgets
    channel('churn').set('by_age_method', None)
    channel('churn').set('probability', None)

channel('db').set('hosts', list(mongo_configs.keys()))
channel('db').watch('selected_host', on_host_selected)

In [ ]:
%%html
<template is="urth-core-bind" channel="db">
    <div class="card-content">
        <paper-dropdown-menu label="Select MongoDB host" 
                selected-item-label="{{ selected_host }}" noink>
            <paper-menu class="dropdown-content" selected="[[ selected_host ]]" 
                attr-for-selected="label">
                <template is="dom-repeat" items="[[ hosts ]]">
                    <paper-item label="[[ item ]]">[[ item ]]</paper-item>
                </template>
            </paper-menu>
        </paper-dropdown-menu>
    </div>
</template>